# SC Cosmetics 

## Import Libaries

In [50]:
!pip install -q --upgrade pip
!pip install -q pyathena
!pip install -q imbalanced-learn
!pip install -q smclarify==0.1
!pip install -q --upgrade statsmodels
!pip install -q --upgrade threadpoolctl
!pip install -q --upgrade s3fs

import boto3
import sagemaker
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pyathena import connect
from imblearn.over_sampling import SMOTE

from smclarify.bias import report
from typing import Dict
from collections import defaultdict

from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score

from sklearn.model_selection import train_test_split

from sqlalchemy import create_engine

%matplotlib inline
%config InlineBackend.figure_format='retina'

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.33.13 which is incompatible.
awscli 1.31.9 requires

## Copy Data from S3

Link to S3 bucket: https://us-east-1.console.aws.amazon.com/s3/buckets/sc-cosmetics?region=us-east-1&bucketType=general&tab=objects

https://sc-cosmetics.s3.amazonaws.com/chemicals-in-cosmetics/cosmetic_chemicals.csv

https://sc-cosmetics.s3.amazonaws.com/cosmetic-ingedients-ranking-pricing/cosmetics.csv

https://sc-cosmetics.s3.amazonaws.com/customer-propensity-to-purchase/customer.csv

In [51]:
# define the S3 bucket, role, and region
sess = sagemaker.Session()
bucket = "sc-cosmetics"  
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# define S3 paths for the source (public) and destination (private) locations
s3_public_path_chemicals = "s3://sc-cosmetics/chemicals-in-cosmetics"
s3_public_path_cosmetics = "s3://sc-cosmetics/cosmetic-ingredients-ranking-pricing"
s3_public_path_customer = "s3://sc-cosmetics/customer-propensity-to-purchase"
s3_private_path = f"s3://{bucket}/"  
s3_private_path_chemicals = f"s3://{bucket}/chemicals-in-cosmetics"
s3_private_path_cosmetics = f"s3://{bucket}/cosmetic-ingredients-ranking-pricing"
s3_private_path_customer = f"s3://{bucket}/customer-propensity-to-purchase"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [52]:
# copy data from the public S3 buckets to our private S3 buckets
!aws s3 cp --recursive $s3_public_path_chemicals/ $s3_private_path_chemicals/
!aws s3 cp --recursive $s3_public_path_cosmetics/ $s3_private_path_cosmetics/
!aws s3 cp --recursive $s3_public_path_customer/ $s3_private_path_customer/

copy: s3://sc-cosmetics/chemicals-in-cosmetics/cosmetic_chemicals.csv to s3://sc-cosmetics/chemicals-in-cosmetics/cosmetic_chemicals.csv
copy: s3://sc-cosmetics/cosmetic-ingredients-ranking-pricing/cosmetics.csv to s3://sc-cosmetics/cosmetic-ingredients-ranking-pricing/cosmetics.csv
copy: s3://sc-cosmetics/customer-propensity-to-purchase/customer.csv to s3://sc-cosmetics/customer-propensity-to-purchase/customer.csv


In [53]:
# list files in private S3 bucket
print("Files in private S3 bucket - Chemicals:")
!aws s3 ls $s3_private_path_chemicals/

print("\nFiles in private S3 bucket - Cosmetics:")
!aws s3 ls $s3_private_path_cosmetics/

print("\nFiles in private S3 bucket - Customer:")
!aws s3 ls $s3_private_path_customer/

Files in private S3 bucket - Chemicals:
2024-04-14 21:25:32   24067011 cosmetic_chemicals.csv

Files in private S3 bucket - Cosmetics:
2024-04-14 21:25:34    1157561 cosmetics.csv

Files in private S3 bucket - Customer:
2024-04-14 21:25:35   35077618 customer.csv


In [54]:
# read files into pd df
chemicals_df = pd.read_csv(s3_private_path_chemicals + "/cosmetic_chemicals.csv")
cosmetics_df = pd.read_csv(s3_private_path_cosmetics + "/cosmetics.csv")
customer_df = pd.read_csv(s3_private_path_customer + "/customer.csv")

## Data Exploration

In [55]:
pd.set_option('display.max_columns', None)

In [56]:
chemicals_df.head()

,ProductName,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,SubCategory,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalCount,Discontinued,ChemicalRemoved,brand_id
0,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),53,Lip Color - Lipsticks Liners and Pencils,6,Titanium dioxide,2009-06-17,2013-08-28,2009-07-09,2009-07-09,1,1,0,70.0
1,Glover's Medicated Shampoo,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,Hair Shampoos (making a cosmetic claim),4,Distillates (coal tar),2009-07-01,2009-07-01,2009-07-01,2009-07-01,2,0,0,960.0
2,Glover's Medicated Shampoo,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,Hair Shampoos (making a cosmetic claim),5,Estragole,2009-07-01,2009-07-01,2009-07-02,2009-07-02,2,0,0,960.0
3,PRECISION GLIMMER EYE LINER-ALL SHADES �,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),46,Eyeliner/Eyebrow Pencils,7,Titanium dioxide,2009-07-09,2013-08-28,2009-07-09,2009-07-09,1,0,0,70.0
4,AVON BRILLIANT SHINE LIP GLOSS-ALL SHADES �,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),52,Lip Gloss/Shine,8,Titanium dioxide,2009-07-09,2013-08-28,2009-07-09,2009-07-09,1,1,0,70.0
5,JILLIAN DEMPSEY FOR AVON CELESTIAL EYESHADOW-A...,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),48,Eye Shadow,9,Titanium dioxide,2009-07-09,2013-08-28,2009-07-09,2009-07-09,1,1,0,70.0
6,ABSOLUTE Precision Color Powder System - All S...,11,OPI PRODUCTS INC.,OPI,59,Nail Products,60,Artificial Nails and Related Products,12178,Titanium dioxide,2009-07-13,2010-12-30,2009-12-04,2009-12-04,1,0,0,1608.0
7,ABSOLUTE Precision Color Powder System - All S...,11,OPI PRODUCTS INC.,OPI,59,Nail Products,60,Artificial Nails and Related Products,12178,Titanium dioxide,2009-07-13,2010-12-30,2009-12-04,2009-12-04,1,0,0,1608.0
8,ABSOLUTE Precision Color Powder System - All S...,11,OPI PRODUCTS INC.,OPI,59,Nail Products,60,Artificial Nails and Related Products,12178,Titanium dioxide,2009-07-13,2010-12-30,2009-12-04,2009-12-04,1,0,0,1608.0
9,ABSOLUTE Precision Color Powder System Opaque ...,11,OPI PRODUCTS INC.,ABSOLUTE,59,Nail Products,60,Artificial Nails and Related Products,17903,Titanium dioxide,2009-07-13,2010-12-30,2010-06-09,2010-06-09,1,0,0,44.0


In [57]:
chemicals_df['PrimaryCategory'].value_counts() 

Makeup Products (non-permanent)      75827
Nail Products                        15347
Skin Care Products                    7683
Sun-Related Products                  4939
Bath Products                         3466
Hair Coloring Products                2061
Hair Care Products (non-coloring)     1620
Tattoos and Permanent Makeup          1477
Personal Care Products                 765
Fragrances                             654
Oral Hygiene Products                  525
Shaving Products                       222
Baby Products                           49
Name: PrimaryCategory, dtype: int64

In [58]:
cosmetics_df['Label'].value_counts()

Moisturizer    298
Cleanser       281
Face Mask      266
Treatment      248
Eye cream      209
Sun protect    170
Name: Label, dtype: int64

In [59]:
customer_df.head()

,UserID,basket_icon_click,basket_add_list,basket_add_detail,sort_by,account_page_click,detail_wishlist_add,list_size_dropdown,closed_minibasket_click,checked_delivery_detail,checked_returns_detail,sign_in,saw_checkout,saw_delivery,saw_homepage,device_mobile,device_computer,device_tablet,returning_user,loc_uk,ordered,product_id
0,a720-6b732349-a720-4862-bd21-644732,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,NaN
1,a0c0-6b73247c-a0c0-4bd9-8baa-797356,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NaN
2,86a8-6b735c67-86a8-407b-ba24-333055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,NaN
3,6a3d-6b736346-6a3d-4085-934b-396834,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,NaN
4,b74a-6b737717-b74a-45c3-8c6a-421140,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,321.0


Since the cosmetics dataframe exclusively comprises skincare items, our analysis will focus solely on skincare products and those associated with sun protection from the chemicals present in the cosmetic dataframe.

## Table Manipulation

### chemicals table

### cosmetics table

### customer table

## Create Athena Database

In [60]:
# athena database name
db_name = 'sc_cosmetics'

In [61]:
# S3 staging directory
s3_staging = "s3://{0}/athena/staging".format(bucket)

In [62]:
# create connection for athena database
conn = connect(region_name=region, s3_staging_dir=s3_staging)

In [63]:
create_query = "CREATE DATABASE IF NOT EXISTS {}".format(db_name)
pd.read_sql(create_query, conn)

""


In [64]:
# create chemicals in cosmetics table
chem_table_name ='cosmetic_chemicals'

# chemical data S3 directory
chem_s3_dir = f's3://{bucket}/chemicals-in-cosmetics'

# drop table if it already exists
drop_query = f'DROP TABLE IF EXISTS {db_name}.{chem_table_name}'
pd.read_sql(drop_query, conn)

# create table
create_query = f"""
                CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.{chem_table_name}(
                ProductName string,
                CompanyID int,
                CompanyName string,
                BrandName string,
                PrimaryCategoryID int,
                PrimaryCategory string,
                SubCategoryID int,
                SubCategory string,
                ChemicalID int,
                ChemicalName string,
                InitialDateReported date,
                MostRecentDateReported date,
                ChemicalCreatedAt date,
                ChemicalUpdatedAt date,
                ChemicalCount int,
                Discontinued int,
                ChemicalRemoved int, 
                brand_id int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{chem_s3_dir}/'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_query, conn)

""


In [65]:
# view top 5 rows of chemical_cosmetics table
# pd.set_option('display.max_rows', None)
pd.read_sql(f'SELECT * FROM {db_name}.{chem_table_name} LIMIT 100', conn)

,productname,companyid,companyname,brandname,primarycategoryid,primarycategory,subcategoryid,subcategory,chemicalid,chemicalname,initialdatereported,mostrecentdatereported,chemicalcreatedat,chemicalupdatedat,chemicalcount,discontinued,chemicalremoved,brand_id
0,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,4.0,New Avon LLC,AVON,44.0,Makeup Products (non-permanent),53,Lip Color - Lipsticks Liners and Pencils,6,Titanium dioxide,2009-06-17,2013-08-28,2009-07-09,2009-07-09,1.0,1.0,0,70
1,Glover's Medicated Shampoo,338.0,J. Strickland & Co.,Glover's,18.0,Hair Care Products (non-coloring),25,Hair Shampoos (making a cosmetic claim),4,Distillates (coal tar),2009-07-01,2009-07-01,2009-07-01,2009-07-01,2.0,0.0,0,960
2,Glover's Medicated Shampoo,338.0,J. Strickland & Co.,Glover's,18.0,Hair Care Products (non-coloring),25,Hair Shampoos (making a cosmetic claim),5,Estragole,2009-07-01,2009-07-01,2009-07-02,2009-07-02,2.0,0.0,0,960
3,PRECISION GLIMMER EYE LINER-ALL SHADES �,4.0,New Avon LLC,AVON,44.0,Makeup Products (non-permanent),46,Eyeliner/Eyebrow Pencils,7,Titanium dioxide,2009-07-09,2013-08-28,2009-07-09,2009-07-09,1.0,0.0,0,70
4,AVON BRILLIANT SHINE LIP GLOSS-ALL SHADES �,4.0,New Avon LLC,AVON,44.0,Makeup Products (non-permanent),52,Lip Gloss/Shine,8,Titanium dioxide,2009-07-09,2013-08-28,2009-07-09,2009-07-09,1.0,1.0,0,70
5,JILLIAN DEMPSEY FOR AVON CELESTIAL EYESHADOW-A...,4.0,New Avon LLC,AVON,44.0,Makeup Products (non-permanent),48,Eye Shadow,9,Titanium dioxide,2009-07-09,2013-08-28,2009-07-09,2009-07-09,1.0,1.0,0,70
6,ABSOLUTE Precision Color Powder System - All S...,11.0,OPI PRODUCTS INC.,OPI,59.0,Nail Products,60,Artificial Nails and Related Products,12178,Titanium dioxide,2009-07-13,2010-12-30,2009-12-04,2009-12-04,1.0,0.0,0,1608
7,ABSOLUTE Precision Color Powder System - All S...,11.0,OPI PRODUCTS INC.,OPI,59.0,Nail Products,60,Artificial Nails and Related Products,12178,Titanium dioxide,2009-07-13,2010-12-30,2009-12-04,2009-12-04,1.0,0.0,0,1608
8,ABSOLUTE Precision Color Powder System - All S...,11.0,OPI PRODUCTS INC.,OPI,59.0,Nail Products,60,Artificial Nails and Related Products,12178,Titanium dioxide,2009-07-13,2010-12-30,2009-12-04,2009-12-04,1.0,0.0,0,1608
9,ABSOLUTE Precision Color Powder System Opaque ...,11.0,OPI PRODUCTS INC.,ABSOLUTE,59.0,Nail Products,60,Artificial Nails and Related Products,17903,Titanium dioxide,2009-07-13,2010-12-30,2010-06-09,2010-06-09,1.0,0.0,0,44


In [79]:
# create subquery with the skin care products and sun-related products 
skinproducts_sub = ["Sun-Related Products", "Skin Care Products"]
in_clause = "', '".join(skinproducts_sub)

sub_query = f"""
            (
            SELECT *
            FROM sc_cosmetics.cosmetic_chemicals
            WHERE PrimaryCategory IN ('{in_clause}')
            )
"""

# create a new table using the subquery
cosmetic_chemical_sub = f"""
                         CREATE TABLE IF NOT EXISTS {db_name}.cosmetic_chemical_sub AS {sub_query}
                         """

# execute the create table query
with conn.cursor() as cursor:
    cursor.execute(cosmetic_chemical_sub)

In [80]:
# view top 5 rows of cosmetic_chemical_sub table
pd.read_sql(f'SELECT * FROM {db_name}.cosmetic_chemical_sub LIMIT 10', conn)

,productname,companyid,companyname,brandname,primarycategoryid,primarycategory,subcategoryid,subcategory,chemicalid,chemicalname,initialdatereported,mostrecentdatereported,chemicalcreatedat,chemicalupdatedat,chemicalcount,discontinued,chemicalremoved,brand_id
0,Nu Colour Tinted Moisturizer Broad Spectrum SP...,248,NSE Products Inc.,Nu Skin,106,Sun-Related Products,107,Sunscreen (making a cosmetic claim),10012,Titanium dioxide,2009-10-20,2013-12-27,2009-10-20,2013-09-17,2,0,1,1590
1,Nu Colour Tinted Moisturizer Broad Spectrum SP...,248,NSE Products Inc.,Nu Skin,106,Sun-Related Products,107,Sunscreen (making a cosmetic claim),33293,Retinyl palmitate,2009-10-20,2013-12-27,2013-12-27,2013-12-27,2,0,0,1590
2,Nature's Gate Organics Advanced Care Natural R...,1232,LEVLAD LLC,Nature's Gate,90,Skin Care Products,91,Acne Products (making a cosmetic claim),10013,Methyleugenol,2009-10-20,2019-07-18,2009-10-20,2009-10-20,1,1,0,1546
3,Nature's Gate Organics Advanced Care Natural R...,1232,LEVLAD LLC,Nature's Gate,90,Skin Care Products,93,Skin Cleansers,10013,Methyleugenol,2009-10-20,2019-07-18,2009-10-20,2009-10-20,1,1,0,1546
4,DDF Wrinkle Resist Plus Pore Minimizer Moistur...,86,The Procter & Gamble Company,DDF,90,Skin Care Products,102,Skin Moisturizers (making a cosmetic claim),10018,Titanium dioxide,2009-10-21,2017-05-11,2009-10-21,2009-10-21,1,1,0,651
5,Lancome Absolue Night Precious Cells,67,L'Oreal USA,Lancome,90,Skin Care Products,97,Nighttime Skin Care,10019,Titanium dioxide,2009-10-21,2009-10-21,2009-10-21,2009-10-21,1,0,0,1287
6,Lancome Renergie Microlift Eye R.A.R.E Intense...,67,L'Oreal USA,Lancome,90,Skin Care Products,161,Facial Cream,10020,Titanium dioxide,2009-10-21,2009-10-21,2009-10-21,2009-10-21,1,0,0,1287
7,Lancome Creme Douceur Cream-To-Oil Massage Cle...,67,L'Oreal USA,Lancome,90,Skin Care Products,93,Skin Cleansers,10021,Titanium dioxide,2009-10-21,2009-10-21,2009-10-21,2009-10-21,1,0,0,1287
8,Lancome High Resolution Eye Refill-3X,67,L'Oreal USA,Lancome,90,Skin Care Products,161,Facial Cream,10023,Titanium dioxide,2009-10-21,2009-10-21,2009-10-21,2009-10-21,1,0,0,1287
9,Lancome Bienfait Multi-Vital Eye SPF 28 Sunscreen,67,L'Oreal USA,Lancome,106,Sun-Related Products,107,Sunscreen (making a cosmetic claim),10024,Titanium dioxide,2009-10-21,2009-10-21,2009-10-21,2009-10-21,1,0,0,1287


In [68]:
# create cosmetics table
cos_table_name ='cosmetics'

# cosmetics data S3 directory
cos_s3_dir = f's3://{bucket}/cosmetic-ingredients-ranking-pricing'

# drop table if it already exists
drop_query = f'DROP TABLE IF EXISTS {db_name}.{cos_table_name}'
pd.read_sql(drop_query, conn)

# create table
create_query = f"""
                CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.{cos_table_name}(
                Label string,
                Brand string,
                Name string,
                Price float,
                Rank float,
                Ingredients string,
                Combination int,
                Dry int,
                Normal int,
                Oily int,
                Sensitive int,
                product_id int, 
                brand_id int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{cos_s3_dir}/'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_query, conn)

""


In [69]:
# view top 5 rows of cosmetics table
pd.read_sql(f'SELECT * FROM {db_name}.{cos_table_name} LIMIT 5', conn)

,label,brand,name,price,rank,ingredients,combination,dry,normal,oily,sensitive,product_id,brand_id
0,Moisturizer,LA MER,Crème de la Mer,175.0,4.1,Algae (Seaweed) Extract- Mineral Oil- Petrolat...,1,1,1,1,1,64,1
1,Moisturizer,SK-II,Facial Treatment Essence,179.0,4.1,Galactomyces Ferment Filtrate (Pitera)- Butyle...,1,1,1,1,1,95,2
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68.0,4.4,Water- Dicaprylyl Carbonate- Glycerin- Ceteary...,1,1,1,1,0,29,3
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175.0,3.8,Algae (Seaweed) Extract- Cyclopentasiloxane- P...,1,1,1,1,1,64,4
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38.0,4.1,Water- Snail Secretion Filtrate- Phenyl Trimet...,1,1,1,1,1,49,5


In [70]:
# create customer table
cust_table_name ='customer'

# customer data S3 directory
cust_s3_dir = f's3://{bucket}/customer-propensity-to-purchase'

# drop table if it already exists
drop_query = f'DROP TABLE IF EXISTS {db_name}.{cust_table_name}'
pd.read_sql(drop_query, conn)

# create table
create_query = f"""
                CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.{cust_table_name}(
                UserID string,
                basket_icon_click int,
                basket_add_list int,
                basket_add_detail int,
                sort_by int,
                account_page_click int,
                detail_wishlist_add int,
                list_size_dropdown int,
                closed_minibasket_click int,
                checked_delivery_detail int,
                checked_returns_detail int,
                sign_in int,
                saw_checkout int,
                saw_delivery int,
                saw_homepage int,
                device_mobile int,
                device_computer int,
                device_tablet int,
                returning_user int,
                loc_uk int,
                ordered int,
                product_id int
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{cust_s3_dir}/'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_query, conn)

""


In [71]:
# view top 5 rows of customer table
pd.read_sql(f'SELECT * FROM {db_name}.{cust_table_name} LIMIT 5', conn)

,userid,basket_icon_click,basket_add_list,basket_add_detail,sort_by,account_page_click,detail_wishlist_add,list_size_dropdown,closed_minibasket_click,checked_delivery_detail,checked_returns_detail,sign_in,saw_checkout,saw_delivery,saw_homepage,device_mobile,device_computer,device_tablet,returning_user,loc_uk,ordered,product_id
0,84d0-82c61917-84d0-41aa-a04c-607845,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,None
1,68dc-82c6b1c9-68dc-b168-8425-91412,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,None
2,8786-82c1eb1b-8786-411b-bd14-548509,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,None
3,1a2b-82c6b429-1a2b-4c1d-b019-845548,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,None
4,c752-82c6b546-c752-47a3-a5bd-104606,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,None


In [72]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,dsoaws
2,sagemaker_featurestore
3,sc_cosmetics


In [81]:
# define query to show tables in db
show_tables_query = f"SHOW TABLES IN {db_name}"

# execute query+ display tables in db
tables_df = pd.read_sql(show_tables_query, conn)
tables_df

,tab_name
0,cosmetic_chemical_sub
1,cosmetic_chemicals
2,cosmetic_chemicals_sub
3,cosmetics
4,customer


In [74]:
if db_name in df_show.values:
    ingest_create_athena_db_passed = True

In [75]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [82]:
# define SQL query
query = """
        SELECT cu.UserID,
               cu.basket_icon_click,
               cu.basket_add_list,
               cu.account_page_click,
               cu.detail_wishlist_add,
               cu.saw_checkout,
               cu.device_mobile,
               cu.device_computer, 
               cu.device_tablet,
               cu.returning_user,
               cu.ordered,
               c.*,
               ccc.PrimaryCategory,
               ccc.SubCategory,
               ccc.ChemicalName,
               ccc.ChemicalCount,
               ccc.Discontinued,
               ccc.ChemicalRemoved
        FROM sc_cosmetics.customer cu
        INNER JOIN sc_cosmetics.cosmetics c ON cu.product_id = c.product_id
        INNER JOIN sc_cosmetics.cosmetic_chemical_sub ccc ON c.brand_id = ccc.brand_id;
        """

In [83]:
sc_df = pd.read_sql(query, conn)

In [84]:
sc_df.sample(n=10, random_state=42)

,UserID,basket_icon_click,basket_add_list,account_page_click,detail_wishlist_add,saw_checkout,device_mobile,device_computer,device_tablet,returning_user,ordered,label,brand,name,price,rank,ingredients,combination,dry,normal,oily,sensitive,product_id,brand_id,PrimaryCategory,SubCategory,ChemicalName,ChemicalCount,Discontinued,ChemicalRemoved
28322,0506-5a94a54a-0506-4727-86b1-799591,1,1,0,0,1,1,0,0,0,1,Sun protect,LAURA MERCIER,Flawless Skin Daily Face Shield SPF 40,45.0,4.2,Water- Cyclomethicone- Butylene Glycol- Ethylh...,0.0,0.0,0,0,0,68,1424,Skin Care Products,Skin Cleansers,Titanium dioxide,1.0,0.0,0.0
76289,0d26-00968570-0d26-4279-bd68-245415,1,1,0,0,1,1,0,0,0,1,Sun protect,TARTE,Tarteguard 20 Tinted Moisturizer Broad Spectru...,36.0,4.0,Water- Cyclopentasiloxane- Peg/Ppg-18/18 Dimet...,1.0,1.0,1,1,1,104,1382,Skin Care Products,Skin Moisturizers (making a cosmetic claim),Titanium dioxide,1.0,0.0,0.0
36941,9437-bd21e968-9437-4373-8768-850887,1,1,0,0,1,1,0,0,1,1,Face Mask,FOREO,Shimmer Freak Mask,19.0,1.0,-Rose Water -Caffeine-infused,1.0,1.0,1,1,1,39,1030,Sun-Related Products,Suntan Enhancers,Titanium dioxide,1.0,0.0,0.0
53778,5614-a201bb73-5614-4899-a3a1-115324,1,0,0,0,1,1,0,1,1,1,Treatment,MURAD,Revitalixir™ Recovery Serum,89.0,4.5,Water- Glycerin- Propanediol- Prunus Armeniaca...,1.0,1.0,1,1,1,76,684,Skin Care Products,Skin Moisturizers (making a cosmetic claim),Vitamin A palmitate,1.0,0.0,0.0
73837,1613-0a49b567-1613-4b82-bc1c-212030,0,0,1,0,1,1,0,0,1,1,Face Mask,DR. JART+,Shake & Shot™ Rubber Masks,12.0,4.1,-Vitamin C (Ascorbic Acid): Brightens the look...,1.0,1.0,1,1,1,28,899,Skin Care Products,Skin Moisturizers (making a cosmetic claim),Titanium dioxide,1.0,0.0,0.0
701,285b-6771e68d-285b-471e-856d-996231,0,0,0,0,1,1,0,0,0,1,Cleanser,KOH GEN DO,Cleansing Spa Water Makeup Remover,46.0,4.5,Water- Dipropylene Glycol- Peg-7 Glyceryl Coco...,0.0,0.0,0,0,0,59,403,Sun-Related Products,Sunscreen (making a cosmetic claim),Titanium dioxide,1.0,1.0,0.0
66364,7651-9807b01b-7651-4154-95c9-844902,1,0,0,0,1,0,0,1,1,1,Face Mask,DR. JART+,Shake & Shot™ Rubber Brightening Mask,12.0,4.3,-Vitamin C (Ascorbic Acid): Brightens the look...,1.0,1.0,1,1,1,28,980,Skin Care Products,Other Skin Care Product,Titanium dioxide,1.0,1.0,0.0
5567,971c-6817702a-971c-4084-8909-645070,1,1,0,0,1,1,0,0,1,1,Eye cream,CLARINS,Eye Contour Gel,41.0,3.8,Water- Rosa Damascena Flower Water- Citrus Aur...,0.0,0.0,0,0,0,17,1235,Skin Care Products,Anti-Wrinkle/Anti-Aging Products (making a cos...,Titanium dioxide,1.0,0.0,0.0
61071,10c0-973471b6-10c0-465c-9c5a-165515,0,1,0,0,1,0,1,0,1,1,Eye cream,DRUNK ELEPHANT,Shaba Complex™ Eye Serum,60.0,3.9,Water- Octyldodecanol- Glycerin- Boron Nitride...,1.0,1.0,1,1,1,29,1106,Skin Care Products,Skin Moisturizers (making a cosmetic claim),Titanium dioxide,1.0,0.0,0.0
26054,2597-ddc51eb1-2597-4628-ab10-693351,1,0,0,0,1,1,0,0,0,1,Face Mask,FRESH,Peony Brightening Night Treatment Mask,82.0,4.0,Water- Glycerin- Caprylic/Capric Triglyceride-...,1.0,1.0,1,1,1,40,923,Skin Care Products,Facial Cream,Titanium dioxide,1.0,0.0,0.0


## Exploratory Data Analysis (EDA)

In [ ]:
# inspect df 
print('Number of Rows:', sc_df.shape[0])
print('Number of Columns:', sc_df.shape[1], '\n')

sc_df.info()

### Identifying and Removing Null Values

In [ ]:
sc_df.isnull().sum()

In [ ]:
# calculate missing values as a percentage of total data 
missing_percentage = (sc_df.isnull().sum() / len(sc_df)) * 100 # convert to percentage 
missing_percentage = missing_percentage.sort_values(ascending=False) # place in ascending order

print('Missing Value Percentage per Column:')
print(missing_percentage)

In [ ]:
# impute values with rows missing less than 20% data using mode 
low_missing_cols = ['ChemicalCount', 'price', 'rank', 'dry','ChemicalRemoved', 'Discontinued']
for col in low_missing_cols:
    sc_df[col] = sc_df[col].fillna(sc_df[col].mode()[0])

In [ ]:
sc_df.isnull().sum() # confrim misisng values are handles 

In [ ]:
sc_df.head()

In [ ]:
sc_df['ChemicalName'].value_counts() # chemicals present in cosmetics 

In [ ]:
sc_df.shape #30 columns and 53325 rows of data 

### Visualizations 

#### Histograms

In [ ]:
# list numerical predictors for histograms
predictors = [
    'account_page_click', 'detail_wishlist_add', 'saw_checkout', 'device_mobile', 
    'device_computer', 'device_tablet', 'ordered', 'price', 'rank', 'product_id', 
    'brand_id', 'ChemicalCount', 'Discontinued', 'ChemicalRemoved'
]

# create figure
num_cols = 4
num_rows = (len(predictors) - 1) // num_cols + 1
plt.figure(figsize=(18, 14))  

# create histograms 
for i, predictor in enumerate(predictors):
    plt.subplot(num_rows, num_cols, i + 1)  
    plt.hist(sc_df[predictor], bins=20, edgecolor='black')
    plt.title(f'Histogram of {predictor}')
    plt.xlabel(predictor)
    plt.ylabel('Frequency')

plt.tight_layout()  
plt.show()

#### Box Plots

In [ ]:
# list numerical predictors to create boxplots 
predictors = numerical_predictors = [
    'account_page_click','detail_wishlist_add', 'saw_checkout', 'device_mobile', 
    'device_computer', 'device_tablet', 'ordered', 'price', 'rank', 'product_id', 
    'brand_id', 'ChemicalCount', 'Discontinued', 'ChemicalRemoved'
]

# create figure for boxplots subplots 
plt.figure(figsize=(18, 14))  

# create boxplots for each predictor
for i, predictor in enumerate(predictors):
    plt.subplot(6, 4, i + 1)  
    sns.boxplot(data=sc_df, y=predictor, width=0.5)
    plt.title(f'Boxplot of {predictor}')
    plt.ylabel(predictor)

plt.tight_layout()  
plt.show()

#### Bar Plots 

In [ ]:
# list of selected columns for bar plot 
selected_columns = [
    'basket_icon_click', 'basket_add_list', 'account_page_click', 
    'detail_wishlist_add', 'saw_checkout', 'device_mobile', 
    'device_computer', 'device_tablet', 'returning_user', 
    'ordered', 'combination', 'dry', 'normal', 'oily', 'sensitive', 
    'ChemicalCount', 'Discontinued', 'ChemicalRemoved','PrimaryCategory', 'label'
]

# calculate the number of rows and columns for the subplot grid
num_cols = 3
num_rows = (len(selected_columns) - 1) // num_cols + 1

# create subplots and flatten axis
fig, axes = plt.subplots(num_rows, num_cols, figsize=(25, 30))
axes = axes.flatten()

# iterate through selected columns and create horizontal countplots
for i, column in enumerate(selected_columns):
    sns.countplot(data=sc_df, y=column, ax=axes[i])
    axes[i].tick_params(axis='x', rotation=45)  

    axes[i].set_title(f'{column}')
    axes[i].set_xlabel('Count')
    axes[i].set_ylabel('')

# remove empty subplots
for i in range(len(selected_columns), num_rows * num_cols):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

#### Class Imbalance 
#### ('ordered' and 'saw_checkout')

In [ ]:
# check distribution of classes before adding class
print("\nBefore balancing:")
print(sc_df['ordered'].value_counts())
print(sc_df['saw_checkout'].value_counts())

In [ ]:
# shuffle df to ensure randomness
sc_shuffled = sc_df.sample(frac=1)
sc_shuffled['ChemicalCount'] = sc_shuffled['ChemicalCount'].astype(int)

# define columns to balance 
columns_to_balance = ['ordered', 'saw_checkout']

# calculate number of records for 'saw_checkout' class 1 to maintain 60% balance
num_records_to_change_saw_checkout = int(len(sc_shuffled[sc_shuffled['saw_checkout'] == 1]) * 0.6)
records_to_change_saw_checkout = sc_shuffled[sc_shuffled['saw_checkout'] == 1].head(num_records_to_change_saw_checkout)
records_to_change_saw_checkout['saw_checkout'] = 0

# update df 
sc_shuffled.loc[records_to_change_saw_checkout.index, 'saw_checkout'] = records_to_change_saw_checkout['saw_checkout']


# define threshold for chemical count to determine class imbalance
chemical_count_threshold = 1

# calculate the number of records to change for 'ordered' class 1 based on chemical count 
num_records_to_change_ordered = int(len(sc_shuffled[(sc_shuffled['ordered'] == 1) & 
                                                    (sc_shuffled['ChemicalCount'] <= chemical_count_threshold)]))

# select records where 'ordered' = 1 and chemical count does not exceed the threshold
records_to_change_ordered = sc_shuffled[(sc_shuffled['ordered'] == 1) & 
                                        (sc_shuffled['ChemicalCount'] <= chemical_count_threshold)].head(num_records_to_change_ordered)
records_to_change_ordered['ordered'] = 0

# update df 
sc_shuffled.loc[records_to_change_ordered.index, 'ordered'] = records_to_change_ordered['ordered']

In [ ]:
# check distribution of classes after adding class
print("\nAfter balancing:")
print(sc_shuffled ['saw_checkout'].value_counts())
print(sc_shuffled ['ordered'].value_counts())

In [ ]:
plt.figure(figsize=(10, 6))

# bar plot for 'ordered' column
plt.bar(['Not Ordered', 'Ordered'], sc_shuffled['ordered'].value_counts(), color='blue', label='Ordered')

# bar plot for 'saw_checkout' column
plt.bar(['Not Checked Out', 'Checked Out'], sc_shuffled['saw_checkout'].value_counts(), color='red', label='Saw Checkout', alpha=0.5)

plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Imbalance After Balancing')
plt.legend()
plt.tight_layout()

plt.show()

In [ ]:
checkout_distribution = sc_shuffled['saw_checkout'].value_counts(normalize=True)
print("Class Distribution for Saw Check Out:\n", checkout_distribution)

ordered_distribution = sc_shuffled['ordered'].value_counts(normalize=True)
print("Class Distribution for Ordered:\n", ordered_distribution)

In [ ]:
balanced_df = sc_shuffled.copy()
balanced_df.info()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# count of ordered by product category
order_count_by_category = balanced_df.groupby('PrimaryCategory')['ordered'].sum()
order_count_by_category.plot(kind='bar', ax=axs[0, 0], 
                             xlabel='Product Category', ylabel='Order Count', 
                             title='Orders by Product Category')

# composition of skin types
skin_type_counts = balanced_df[['dry', 'normal', 'oily', 'sensitive']].sum()
skin_type_counts.plot(kind='bar', stacked=True, ax=axs[0, 1], 
                      xlabel='Skin Types', ylabel='Count', 
                      title='Composition of Skin Types')

# device usage 
device_usage = balanced_df[['device_computer', 'device_tablet', 'device_mobile']].astype(int)
device_usage_counts = device_usage.groupby(device_usage.columns.tolist()).size().unstack(fill_value=0)
sns.heatmap(device_usage_counts, ax=axs[0, 2], cmap="Blues", annot=True, fmt="d",
            cbar_kws={'label': 'Count'}, 
            linewidths=0.5, linecolor='black')
axs[0, 2].set_title('Device Usage')

# returning vs new users 
returning_new_users = pd.crosstab(balanced_df['returning_user'], 
                                  balanced_df['ordered'])

returning_new_users.plot(kind='bar', stacked=True, ax=axs[1, 0], 
                         xlabel='User Type', ylabel='Count', 
                         title='Returning vs. New Users')

# distribution of ratings 
sns.histplot(data=balanced_df, x='rank', bins=10, kde=True, ax=axs[1, 1])
axs[1, 1].set_xlabel('Rating')
axs[1, 1].set_ylabel('Frequency')
axs[1, 1].set_title('Distribution of Ratings')

# discontinued products 
discontinued_category_counts = pd.crosstab(index=balanced_df['Discontinued'], 
                                           columns=balanced_df['PrimaryCategory'])

discontinued_category_counts.plot(kind='bar', stacked=True, ax=axs[1, 2], 
                                  xlabel='Discontinued', ylabel='Count', 
                                  title='Discontinued Products by Category')

plt.tight_layout()
plt.show()

In [ ]:
#  order count by brand and product category
order_count_by_brand_category = balanced_df.groupby(['brand', 'PrimaryCategory'])['ordered'].sum().unstack().fillna(0)
order_count_by_brand = order_count_by_brand_category.sum(axis=1).sort_values()
order_count_by_brand_category = order_count_by_brand_category.loc[order_count_by_brand.index]


order_count_by_brand_category.plot(kind='barh', stacked=True, figsize=(20, 20))
plt.xlabel('Order Count')
plt.ylabel('Product Category')
plt.title('Orders by Brand and Product Category')
plt.legend(title='Brand', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# plot the average price by brand
avg_price_by_brand = balanced_df.groupby('brand')['price'].mean().sort_values()

plt.figure(figsize=(20, 20))
avg_price_by_brand.plot(kind='barh')
plt.xlabel('Brand')
plt.ylabel('Average Price')
plt.title('Average Price by Brand')
plt.tight_layout()
plt.show()

In [ ]:
# number of unique chemicals per brand
chemicals_by_brand = balanced_df.groupby('brand')['ChemicalName'].nunique().sort_values()
plt.figure(figsize=(20, 20))
chemicals_by_brand.plot(kind='barh')
plt.xlabel('Number of Unique Chemicals')
plt.ylabel('Brand')
plt.title('Number of Unique Chemicals by Brand')
plt.tight_layout()
plt.show()

In [ ]:
# pivot table to aggregate the binary values for each chemical and skin type combination
pivot_table = balanced_df.pivot_table(index='ChemicalName', 
                                values=['dry', 'normal', 'oily', 'sensitive'], 
                                aggfunc='sum')

In [ ]:
# calculate total number of chemicals per skin type
pivot_table['total'] = pivot_table.sum(axis=1)

# calculate percentage of each skin type for each chemical
for col in pivot_table.columns:
    pivot_table[col] = pivot_table[col] / pivot_table['total'] * 100
pivot_table.drop(columns='total', inplace=True)

# set figure size
plt.figure(figsize=(18, 10))

# plot % of skin types for each chemical 
pivot_table.plot(kind='barh', stacked=True, figsize=(18, 10))
plt.xlabel('Chemical Name')
plt.ylabel('Percentage of Skin Types')
plt.title('Percentage of Skin Types for Each Chemical')

plt.legend(title='Skin Type', loc='upper right')
plt.show();

In [ ]:
# count occurrences of each chemical
chemical_counts = balanced_df['ChemicalName'].value_counts()
chemical_counts_sorted = chemical_counts.sort_values(ascending=True)

# plot bar chart
plt.figure(figsize=(12, 8))
chemical_counts_sorted.plot(kind='barh')
plt.title('Occurrences of Chemicals')
plt.xlabel('Count')
plt.ylabel('Chemical Name')
plt.tight_layout()
plt.show()

In [ ]:
# summary statistics of target column 'ordered'
balanced_df['ordered'].describe()

In [ ]:
# create df for top chemicals + whether they were bought 
top_n_chemicals = balanced_df['ChemicalName'].value_counts().nlargest(10).index
top_n_df = balanced_df[balanced_df['ChemicalName'].isin(top_n_chemicals)]

# set figure
plt.figure(figsize=(20, 10))
sns.countplot(data=top_n_df, x="ChemicalName", hue="ordered")
plt.show()

In [ ]:
plt.figure(figsize=(18, 10))

# relationship between returning vs. new users and ordered status
plt.subplot(2, 2, 1)
sns.countplot(data=balanced_df, x='returning_user', hue='ordered')
plt.title('Relationship between Returning vs. New Users and Ordered Status')

# distribution of ratings for ordered and non-ordered products
plt.subplot(2, 2, 2)
sns.histplot(data=balanced_df, x='rank', hue='ordered', kde=True)
plt.title('Distribution of Ratings for Ordered and Non-Ordered Products')

plt.tight_layout()
plt.show()

In [ ]:
correlation_matrix = balanced_df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

### Data Scrubbing 

#### Data Consistency

In [ ]:
# data consitency w/device columns 
inconsistent_records = balanced_df[(balanced_df['device_mobile'] + balanced_df['device_computer'] + balanced_df['device_tablet']) > 1]
print("Inconsistent records:", inconsistent_records)

In [ ]:
# drop inconcistent records - rows where the sum of device columns is greater than 1 (1049 rows were picked up to have suplicate values)
inconsistent_records_index = (balanced_df['device_mobile'] + balanced_df['device_computer'] + balanced_df['device_tablet']) > 1
balanced_df.drop(inconsistent_records_index.index[inconsistent_records_index], inplace=True)

In [ ]:
balanced_df.shape #shape after duplicate values were dropped 

#### Data Type Conversion 

In [ ]:
balanced_df.info()

In [ ]:
# convert Rank, Discontinued, and ChemicalRemoved, Dry to integer type
balanced_df['rank'] = balanced_df['rank'].astype(int)
balanced_df['Discontinued'] = balanced_df['Discontinued'].astype(int)
balanced_df['ChemicalRemoved'] = balanced_df['ChemicalRemoved'].astype(int)
balanced_df['dry'] = balanced_df['dry'].astype(int)

#### Outlier Detection 

In [ ]:
# visulaize potential outliers 
def visualize_distribution(balanced_df, columns):
    fig, axes = plt.subplots(nrows=1, ncols=len(columns), figsize=(16, 5))
    for i, col in enumerate(columns):
        sns.histplot(balanced_df[col], ax=axes[i], kde=True)
        axes[i].set_title(col)
        axes[i].set_xlabel('')
    plt.tight_layout()
    plt.show()

# define numerical columns for visualization
numerical_columns = ['price', 'rank', 'ChemicalCount', 'Discontinued', 'ChemicalRemoved']
visualize_distribution(balanced_df, numerical_columns)

In [ ]:
 balanced_df[numerical_columns].describe() #outliers will remain in the df 

In [ ]:
balanced_df['ChemicalCount'].value_counts() # inspect chemical count distribution 

#### Duplicate Values 

In [ ]:
# check for duplicates
balanced_df.duplicated().sum() #original df is 52276 rows 

In [ ]:
duplicate_rows = balanced_df[balanced_df.duplicated()]
duplicate_rows.head()

In [ ]:
duplicate_rows['UserID'].value_counts() #UserID has duplicate values, rows will be dropped

In [ ]:
# remove duplicate rows 
clean_df = balanced_df.drop_duplicates()
clean_df = clean_df.reset_index(drop=True)

In [ ]:
# inspect df 
print('Number of Rows:', clean_df.shape[0])
print('Number of Columns:', clean_df.shape[1], '\n')

clean_df.info()

In [ ]:
clean_df.head()

### Feature Selection 

#### Chi-Square 

We will use chi-square to identify columns with high p-values and remove the redundant/irrelevant columns. 

In [ ]:
# define target column to be analyzed for assocation 
target_column = 'ordered'

# create an empty array to store the results of chi-square test
chi_results = []

# iterate through all columns (excluding the target column)
for column in clean_df.columns:
    if column != target_column:
        # create a contingency table for the independent and target variable
        cross_tab = pd.crosstab(index=clean_df[column], columns=clean_df[target_column])
        # perform chi-square test
        chi2, p, dof, expected = chi2_contingency(cross_tab)
        # append results to the list
        chi_results.append({
            'Variable': column,
            'Chi-Square Value': chi2,
            'P-Value': p,
            'Degrees of Freedom': dof
        })


# display results in a df 
chi_df = pd.DataFrame(chi_results)
chi_df

In [ ]:
# drop redundant columns such as userid, product_id, and brand_id and those identified with non-signifcant p-values
clean_df.drop(['UserID', 'name', 'brand_id', 'PrimaryCategory', 'SubCategory', 'ingredients', 
               'basket_icon_click', 'basket_add_list', 'ChemicalRemoved', 'ChemicalCount'], axis=1, inplace=True)

#### Variance Inflation Factor (VIF)

In [ ]:
X_numeric = clean_df.select_dtypes(include=['float64', 'int64'])

# create a df for VIF values
vif = pd.DataFrame()
vif["Variable"] = X_numeric.columns

# calculate/store VIF values for each numeric variable
vif["VIF"] = [variance_inflation_factor(X_numeric.values, i) for i in range(X_numeric.shape[1])]
print(vif)

It's recommended to drop variables with high VIF values (>10) as they indicate significant multicollinearity. The following columns will be dropped: device_mobile, device_computer, device_tablet. 

In [ ]:
# drop specified columns
clean_df.drop(['device_mobile', 'device_computer', 'device_tablet'], axis=1, inplace=True)

In [ ]:
# create correlation matrix with reduced df 
correlation_matrix = clean_df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# correlation bewteen ordered and other variables 
ordered_correlation = correlation_matrix['ordered']
ordered_correlation

#### Feature Transformation

In [ ]:
# one hot encode categorical variables
cat_col = ["label", "brand", "ChemicalName", "rank"]
clean_df['ChemicalName'] = clean_df['ChemicalName'].str.replace('"', '')
encoded_df = pd.get_dummies(clean_df, columns=cat_col)

In [ ]:
encoded_df.head()

In [ ]:
encoded_df.shape

#### PCA

In [ ]:
# create a copy of encoded_df before applying PCA 
pca_df = encoded_df.copy()

In [ ]:
# standardize the data
scaler = StandardScaler()
# fit and transform data using scaler 
df_standardized = scaler.fit_transform(pca_df)

In [ ]:
# apply PCA
pca = PCA()
pca.fit(df_standardized)

# calculate the cumulative explained variance ratio
explained_variance_ratio_cumulative = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# plot cumulative explained variance ratio
plt.plot(range(1, len(explained_variance_ratio_cumulative) + 1), explained_variance_ratio_cumulative, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio vs. Number of Components')
plt.show()

In [ ]:
# find the number of components that fall within 80% to 90% cumulative explained variance
target_variance_range = (0.80, 0.90)

# use NumPy to find the index of the first component within the specified variance range
target_components = np.where((explained_variance_ratio_cumulative >= target_variance_range[0]) &
                             (explained_variance_ratio_cumulative <= target_variance_range[1]))[0][0] + 1

print("Ideal number of components within 80% to 90% variance range:", target_components)

In [ ]:
# extract target variable
y = encoded_df['ordered']
X = encoded_df.drop(columns=['ordered'])

# standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# apply PCA
pca = PCA(n_components=97)  
X_pca = pca.fit_transform(X_scaled)

In [ ]:
# concatenate transformed features with target variable
pca_df= pd.DataFrame(X_pca, columns=[f'pca_{i}' for i in range(X_pca.shape[1])])
pca_df['ordered'] = y.values

In [ ]:
pca_df.head()

### Test, Validation, Train Split

In [ ]:
# split 90% modeling and 10 % holdout for prediction
df_train, df_holdout= train_test_split(pca_df, test_size=0.10, train_size=0.90, stratify=pca_df['ordered'])

# split the 5% into half for validation and half for testing
df_val, df_test = train_test_split(df_holdout, test_size=0.5, train_size=0.5, stratify=df_holdout['ordered'])

In [ ]:
df_train.shape

In [ ]:
df_val.shape

In [ ]:
df_test.shape

### Transfer Data to S3 Bucket

In [ ]:
# upload transformed data
# save DataFrame to a CSV file
pca_df.to_csv('sc_data.csv', index=False)

# upload file to S3 bucket
s3 = boto3.client('s3')
s3.upload_file('sc_data.csv', bucket, 'clean-data/sc_data.csv')

In [ ]:
# upload train data
# save DataFrame to a CSV file
df_train.to_csv('train_data.csv', index=False)

# upload file to S3 bucket
s3 = boto3.client('s3')
s3.upload_file('train_data.csv', bucket, 'train-data/train_data.csv')

In [ ]:
# upload validation data
# save DataFrame to a CSV file
df_val.to_csv('validation_data.csv', index=False)

# upload file to S3 bucket
s3 = boto3.client('s3')
s3.upload_file('validation_data.csv', bucket, 'validation-data/validation_data.csv')

In [ ]:
# upload test data
# save DataFrame to a CSV file
df_test.to_csv('test_data.csv', index=False)

# upload file to S3 bucket
s3 = boto3.client('s3')
s3.upload_file('test_data.csv', bucket, 'test-data/test_data.csv')

## Modeling

In [ ]:
!pip install -q autogluon
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular import TabularDataset, TabularPredictor
from tabulate import tabulate

In [ ]:
# specify locations for train and test data
data_root = 's3://sc-cosmetics/'
train_data = TabularDataset(data_root + 'train-data/train_data.csv')
test_data = TabularDataset(data_root + 'test-data/test_data.csv')
model_directory = 's3://sc-cosmetics/autogloun-models' 

# fit models
predictor = TabularPredictor(label='ordered', path=model_directory).fit(train_data=train_data, verbosity=2)

# predictions for the best model
predictions = predictor.predict(test_data)

The best model is WeightedEnsemble_L2. 

In [ ]:
%store predictions

In [ ]:
# get information about the models used in the predictor
predictor.leaderboard(test_data)

In [ ]:
# define predictor path 
local_model_directory = predictor.path 

In [ ]:
%store local_model_directory

In [ ]:
# load predictor to specified path 
predictor = TabularPredictor.load(predictor.path)
predictor 

In [ ]:
# filter for top model (WeightedEnsemble_L2)
leaderboard = predictor.leaderboard(extra_info=True)
weighted_ensemble_row = leaderboard[leaderboard['model'] == 'WeightedEnsemble_L2']

# extract hyperparameters 
we2_hyperparameters = weighted_ensemble_row['hyperparameters'].iloc[0]

# print hyperparameters
print("Hyperparameters of WeightedEnsemble_L2 model:")
print(we2_hyperparameters)

S3 URI to AutoGloun models in S3: s3://sc-cosmetics/autogluon-models/

S3 URI to AutoML models created in SagemakerStudio stored in S3: s3://sc-cosmetics/automl/automl-sc-cosmetics/

### Performance Metrics

In [ ]:
# compute performance metrics
performance = predictor.evaluate(test_data)
performance['ROC AUC'] = performance.pop('roc_auc') # add AUC ROC score

# print the performance metrics
df_performance = pd.DataFrame.from_dict(performance, orient='index', columns=['Value'])

# convert into tabular format
print(tabulate(df_performance, headers='keys', tablefmt='pipe'))

#### Confusion Matrix 

In [ ]:
# confusion matrix
conf_matrix = confusion_matrix(test_data['ordered'], predictions)

# plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap = 'RdBu', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

#### ROC Curve AUC 

In [ ]:
# get predictions
y_probs = predictor.predict_proba(test_data)[1] 
y_true = test_data['ordered'].values

In [ ]:
# compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, y_probs)
auc_score = roc_auc_score(y_true, y_probs)

# plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (AUC = {:.4f})'.format(auc_score))
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line representing random guessing
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.grid(True)
plt.show()

#### Precision Recall Curve

In [ ]:
# precision recall curve
precision, recall, _ = precision_recall_curve(y_true, y_probs)
average_precision = average_precision_score(y_true, y_probs)

# plot precision recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'Precision-Recall curve (AP = {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)
plt.show()

#### Gain Curve

In [ ]:
# gain curve
gains = np.cumsum(np.sort(y_probs)[::-1]) / np.sum(y_probs)
percentile = np.arange(0, len(gains)) / len(gains) * 100

# plot gain curve
plt.figure(figsize=(8, 6))
plt.plot(percentile, gains, label='Gain Curve')
plt.xlabel('Percentage of instances (descending by predicted probability)')
plt.ylabel('Cumulative gains')
plt.title('Gain Curve')
plt.legend()
plt.grid(True)
plt.show()